In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import glob

import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as T
from torchvision.models import resnet50
from torchsummary import summary

In [2]:
dataset_path = '/home/hari/data/Personal/flower_photos'
classes = glob.glob(os.path.join(dataset_path, '**'))
images = glob.glob(os.path.join(dataset_path, '**/*.jpg'))
print(len(classes))
print(len(images))

5
3670


In [3]:
for c in classes:
    images = glob.glob(os.path.join(dataset_path, c, '*.jpg'))
    print(len(images))

641
699
633
799
898


In [4]:
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224), antialias=False),
])

In [5]:
class Dataset(data.Dataset):
    def __init__(self, root_directory, transforms=None, train=True):
        self.root_directory = root_directory
        self.transforms = transforms
        self.train_size = 0.8
        self.train = train
        self.images = self.__train_validation_split__()
        classes = [c.split('/')[-1] for c in sorted(glob.glob(os.path.join(root_directory, '**')))]
        self.class_dict = {c : i for i, c in enumerate(classes)}

    def __train_validation_split__(self):
        images = glob.glob(os.path.join(self.root_directory, '**/*.jpg'))
        _size = int(len(images) * self.train_size) if self.train else int(len(images) * (1 - self.train_size))
        indices = np.random.choice(len(images), _size, replace=False)
        images = np.array(images)[indices]
        return images

    def __class_tonumeric__(self, label):
        return self.class_dict[label]
    
    def __getitem__(self, index):
        path = self.images[index]
        label = path.split('/')[-2]
        label = self.__class_tonumeric__(label)
        image = cv.imread(path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image)
        return image, label
    
    def __len__(self):
        return len(self.images)

In [6]:
dataset = Dataset(root_directory=dataset_path, transforms=transforms, train=True)
validset = Dataset(root_directory=dataset_path, transforms=transforms, train=False)
dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)
validloader = data.DataLoader(validset, batch_size=1, shuffle=True, drop_last=True)
X, y = next(iter(dataloader))
print(X.shape, y.shape)

torch.Size([4, 3, 224, 224]) torch.Size([4])


In [7]:
class LearnedReLU(nn.Module):
    def __init__(self, learning_epochs):
        super().__init__()
        self.learning_epochs = learning_epochs
        positive_coefficient = torch.normal(torch.ones(1), torch.ones(1))
        negative_coefficient = torch.normal(torch.zeros(1), torch.ones(1))
        self.positive_coefficient = nn.Parameter(positive_coefficient)
        self.negative_coefficient = nn.Parameter(negative_coefficient)

    def forward(self, X, epoch=None):
        _p_copy = self.positive_coefficient.data.clone()
        _n_copy = self.negative_coefficient.data.clone()
        if epoch is None or epoch >= self.learning_epochs:
            _p_copy, _n_copy = nn.Parameter(torch.ones(1)), nn.Parameter(torch.zeros(1))
        out = torch.where(X>=0, _p_copy * X, _n_copy * X)
        return out

class PerturbedReLU(nn.Module):
    def __init__(self, epsilon=0.01):
        super().__init__()
        self.epsilon = epsilon
    
    def forward(self, X):
        activated = F.relu(X)
        perturbation = torch.normal(torch.ones(X.shape)*self.epsilon, torch.ones(X.shape)*self.epsilon)
        return activated + perturbation

In [8]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(7, 7), stride=(1, 1), padding=(0, 0))
        self.bn0 = nn.BatchNorm2d(num_features=self.conv0.out_channels)
        self.conv1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(num_features=self.conv1.out_channels)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(num_features=self.conv2.out_channels)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn3 = nn.BatchNorm2d(num_features=self.conv3.out_channels)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn4 = nn.BatchNorm2d(num_features=self.conv4.out_channels)
        self.conv5 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn5 = nn.BatchNorm2d(num_features=self.conv5.out_channels)
        self.conv6 = nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn6 = nn.BatchNorm2d(num_features=self.conv6.out_channels)
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d((3, 3), stride=(1, 1))
        self.flatten = nn.Flatten()
    
    def forward(self, X):
        out = self.activation(self.bn0(self.conv0(X)))
        out = self.activation(self.bn1(self.conv1(out)))
        out = self.activation(self.bn2(self.conv2(out)))
        out = self.pool(out)
        out = self.activation(self.bn3(self.conv3(out)))
        out = self.activation(self.bn4(self.conv4(out)))
        out = self.pool(out)
        out = self.activation(self.bn5(self.conv5(out)))
        out = self.activation(self.bn6(self.conv6(out)))
        out = self.pool(out)
        out = self.flatten(out)
        return out

In [13]:
def pto_balance(initial_balance=78.71, num_periods=1):
    return initial_balance + (6.67 * num_periods)

def take_pto(initial_balance, hours):
    return initial_balance - hours

mid_june_balance = pto_balance(num_periods=6)
late_june_seattle_pto_balance = take_pto(mid_june_balance, 80)
print(late_june_seattle_pto_balance)

38.72999999999999
